<a href="https://colab.research.google.com/github/MarkusThill/BitBully/blob/master/notebooks/getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 BitBully: Getting Started

This notebook introduces the main building blocks of **BitBully**:

- `Board`: represent and manipulate Connect Four positions
- `BitBully`: analyze positions and choose strong moves (optionally with an opening book)

All examples are designed to be copy-pasteable and easy to adapt for your own experiments.


## 📦 Imports

We import BitBully once and reuse the `bb` alias throughout the notebook.


In [ ]:
# Import the BitBully module, which provides a high-performance Connect-4 engine.
# The module includes the Board class, solvers, and helper functions for analysis.
import bitbully as bb

## 🧰 Small helper for nicer output

A tiny helper that prints a label + board (without repeating boilerplate).
You can remove this cell if you prefer raw `print(board)` everywhere.

In [ ]:
def show(title: str, board: bb.Board) -> None:
    """Print a label and a board in a consistent, notebook-friendly way.

    This is a small convenience helper to avoid repeating:

        print("Some title")
        print(board)

    across many cells.

    Args:
        title: A short label shown before the board.
        board: The board position to print.

    Example:
        ```python
        import bitbully as bb

        b = bb.Board("333")
        show("My position:", b)
        ```
        Expected output:
        ```text
        My position:
        _  _  _  _  _  _  _
        _  _  _  _  _  _  _
        _  _  _  _  _  _  _
        _  _  _  X  _  _  _
        _  _  _  O  _  _  _
        _  _  _  X  _  _  _
        ```
    """
    print(title)
    print(board)

## 🧩 Creating Boards: Empty positions

There are multiple ways to create an empty board:

- `Board()` (recommended)
- `Board([[0] * 6 for _ in range(7)])` (column-major layout)
- `Board([[0] * 7 for _ in range(6)])` (row-major layout)

All are equivalent and produce the same empty position.


In [ ]:
# Create an empty board using the default constructor.
board = bb.Board()  # Starts with no tokens placed.

# Alternatively, initialize the board explicitly from a 2D list.
# Each inner list represents a column (7 columns total, 6 rows each).
# A value of 0 indicates an empty cell; 1 and 2 would represent player tokens.
board = bb.Board([[0] * 6 for _ in range(7)])  # Equivalent to an empty board.

# You can also set up a specific board position manually using a 6 x 7 layout,
# where each inner list represents a row instead of a column.
# (Both layouts are accepted by BitBully for convenience.)
# For more complex examples using 2D arrays, see the examples below.
board = bb.Board([[0] * 7 for _ in range(6)])  # Also equivalent to an empty board.

# Display the board in text form.
# The __repr__ method shows the current state (useful for debugging or interactive use).
board

## 🧩 Creating Boards: From a move sequence (list of ints)

A move list like `[3, 3, 3]` means:

- play column 3 three times,
- alternating players automatically (`X` then `O` then `X`).


In [ ]:
# Initialize a board with a sequence of moves played in the center column.

# The list [3, 3, 3] represents three moves in column index 3 (zero-based).
# Moves alternate automatically between Player 1 (yellow, X) and Player 2 (red, O).
# After these three moves, the center column will contain:
#   - Row 0: Player 1 token (bottom)
#   - Row 1: Player 2 token
#   - Row 2: Player 1 token
board = bb.Board([3, 3, 3])

# Display the resulting board.
# The textual output shows the tokens placed in the center column.
show("Board([3, 3, 3]):", board)

## 🧩 Creating Boards: From a compact move string

A move string like `"33333111"` is shorthand for the column sequence:

`3 3 3 3 3 1 1 1`

This is convenient for reproducing positions quickly.


In [ ]:
# Initialize a board using a compact move string.

# The string "33333111" represents a sequence of eight moves:
#   3 3 3 3 3 → five moves in the center column (index 3)
#   1 1 1 → three moves in the second column (index 1)
#
# Moves are applied in order, alternating automatically between Player 1 (yellow, X)
# and Player 2 (red, O), just as if you had called `board.play()` repeatedly.
#
# This shorthand is convenient for reproducing board states or test positions
# without having to provide long move lists.

board = bb.Board("33333111")

# Display the resulting board.
# The printed layout shows how the tokens stack in each column.
show("Board('33333111'):", board)

## 🧩 Creating Boards: From a 2D array (row-major, 6×7)

Row-major format means:

- 6 inner lists (rows)
- each row has 7 entries (columns)
- top row first, bottom row last

Values:
- `0` = empty
- `1` = Player 1 (`X`)
- `2` = Player 2 (`O`)

In [ ]:
# Use a 6 x 7 list (rows x columns) to set up a specific board position manually.

# Each inner list represents a row of the Connect-4 grid.
# Convention:
#   - 0 → empty cell
#   - 1 → Player 1 token (yellow, X)
#   - 2 → Player 2 token (red, O)
#
# The top list corresponds to the *top row* (row index 5),
# and the bottom list corresponds to the *bottom row* (row index 0).
# This layout matches the typical visual display of the board.

board_array = [
    [0, 0, 0, 0, 0, 0, 0],  # Row 5 (top)
    [0, 0, 0, 1, 0, 0, 0],  # Row 4: Player 1 token in column 3
    [0, 0, 0, 2, 0, 0, 0],  # Row 3: Player 2 token in column 3
    [0, 2, 0, 1, 0, 0, 0],  # Row 2: tokens in columns 1 and 3
    [0, 1, 0, 2, 0, 0, 0],  # Row 1: tokens in columns 1 and 3
    [0, 2, 0, 1, 0, 0, 0],  # Row 0 (bottom): tokens stacked lowest
]

# Create a Board instance directly from the 2D list.
# This allows reconstructing arbitrary positions (e.g., from test data or saved states)
# without replaying the move sequence.
board = bb.Board(board_array)

# Display the resulting board state in text form.
board

## 🧩 Creating Boards: From a 2D array (column-major, 7×6)

Column-major format means:

- 7 inner lists (columns)
- each column has 6 entries (rows)
- left column first
- row index `0` is bottom

In [ ]:
# Use a 7 x 6 list (columns x rows) to set up a specific board position manually.

# Each inner list represents a **column** of the Connect-4 board, from left (index 0)
# to right (index 6). Each column contains six entries — one for each row, from
# bottom (index 0) to top (index 5).
#
# Convention:
#   - 0 → empty cell
#   - 1 → Player 1 token (yellow, X)
#   - 2 → Player 2 token (red, O)

board_array = [
    [0, 0, 0, 0, 0, 0],  # Column 0 (leftmost)
    [2, 1, 2, 0, 0, 0],  # Column 1
    [0, 0, 0, 0, 0, 0],  # Column 2
    [1, 2, 1, 2, 1, 0],  # Column 3 (center)
    [0, 0, 0, 0, 0, 0],  # Column 4
    [0, 0, 0, 0, 0, 0],  # Column 5
    [0, 0, 0, 0, 0, 0],  # Column 6 (rightmost)
]

# Create a Board instance directly from the 2D list.
# This allows reconstructing any arbitrary position (e.g., test cases, saved games)
# without replaying all moves individually.
board = bb.Board(board_array)

# Display the resulting board.
# The text output shows tokens as they would appear in a real Connect-4 grid.
board

## 🔄 Resetting an existing board (override its state)

`reset_board(...)` clears the current board and sets a new state.

Here we:
1. create a temporary board,
2. then override it with `[3, 3, 3]`.


In [ ]:
import bitbully as bb

# This is just a temporary setup; it will be replaced below.
board = bb.Board("0123456")

# Set the board state directly from a move sequence.
# The list [3, 3, 3] represents three consecutive moves in the center column (index 3).
# Moves alternate automatically between Player 1 (yellow) and Player 2 (red).
#
# The `set_board()` method clears the current position and replays the given moves
# from an empty board — effectively overriding any existing board state.
assert board.reset_board([3, 3, 3])

# Display the updated board to verify the new position.
board

## 🔄 Resetting back to an empty board

Calling `reset_board()` with no arguments resets to the empty starting position.


In [ ]:
import bitbully as bb

# This is just a temporary setup;
board = bb.Board("0123456")

# reset the board to an empty state
assert board.reset_board()
board

## 🔄 Resetting with different input formats

`reset_board(...)` accepts:

- list of moves: `[3, 3, 3]`
- string of moves: `"33333111"`
- 2D row-major array (6×7)
- 2D column-major array (7×6)

This cell demonstrates each format and ends by printing the final position.

In [ ]:
# Briefly demonstrate the different input formats accepted by `set_board()`.
import bitbully as bb

# Create an empty board instance
board = bb.Board()

# Variant 1: From a list of moves (integers)
# Each number represents a column index (0-6); moves alternate between players.
assert board.reset_board([3, 3, 3])

# Variant 2: From a compact move string
# Equivalent to the list above — useful for quick testing or serialization.
assert board.reset_board("33333111")

# Variant 3: From a 2D list in row-major format (6 x 7)
# Each inner list represents a row (top to bottom).
# 0 = empty, 1 = Player 1, 2 = Player 2.
board_array = [
    [0, 0, 0, 0, 0, 0, 0],  # Row 5 (top)
    [0, 0, 0, 1, 0, 0, 0],  # Row 4
    [0, 0, 0, 2, 0, 0, 0],  # Row 3
    [0, 2, 0, 1, 0, 0, 0],  # Row 2
    [0, 1, 0, 2, 0, 0, 0],  # Row 1
    [0, 2, 0, 1, 0, 0, 0],  # Row 0 (bottom)
]
assert board.reset_board(board_array)

# Variant 4: From a 2D list in column-major format (7 x 6)
# Each inner list represents a column (left to right); this matches BitBully's internal layout.
board_array = [
    [0, 0, 0, 0, 0, 0],  # Column 0 (leftmost)
    [2, 1, 2, 1, 0, 0],  # Column 1
    [0, 0, 0, 0, 0, 0],  # Column 2
    [1, 2, 1, 2, 1, 0],  # Column 3 (center)
    [0, 0, 0, 0, 0, 0],  # Column 4
    [2, 1, 2, 0, 0, 0],  # Column 5
    [0, 0, 0, 0, 0, 0],  # Column 6 (rightmost)
]
assert board.reset_board(board_array)

# Display the final board state in text form
board

## 🎮 Playing moves: list input

`play([3, 3, 3])` applies three moves into the center column, alternating players automatically.

In [ ]:
# Play a sequence of moves into the center column (column index 3)
import bitbully as bb

board = bb.Board()
assert board.play([3, 3, 3])
board

## 🎮 Playing moves: sequence input (`range(7)`)

`play(range(7))` plays one token in each column (0 through 6).

In [ ]:
# Play a sequence of moves across all columns
board = bb.Board()
assert board.play(range(7))
board

## 🎮 Playing moves: string input

`play("33333111")` is a compact shorthand for repeated column plays.

In [ ]:
# Play a sequence defined by a string
board = bb.Board()
assert board.play("33333111")
board

## 🧬 Copying Boards (and keeping the original unchanged)

When analyzing game trees you often want to "try a move" without mutating the current board.
BitBully supports two clean patterns:

- `board.copy()` → returns a *separate* board object with the same position.
- `board.play_on_copy(col)` → returns a *new* board with that move applied (original stays unchanged).

This is perfect for:
- minimax/negamax exploration,
- generating candidate positions,
- testing move ordering,
- or simply keeping notebooks tidy.


In [ ]:
import bitbully as bb

# Start from a known position.
board = bb.Board("33333111")
print("Original board:")
print(board)

# 1) Copy the board (same position, different object).
board_copy = board.copy()
assert board_copy == board
assert board_copy is not board
assert board_copy.uid() == board.uid()

# Mutate only the copy.
assert board_copy.play(0)
assert board_copy != board
assert board_copy.uid() != board.uid()

print("\nModified COPY (played column 0):")
print(board_copy)

print("\nOriginal remains unchanged:")
print(board)

# 2) Functional style: play a move on a new board without mutating.
b2 = bb.Board("333")  # some existing position
b3 = b2.play_on_copy(4)

assert b2.count_tokens() == 3
assert b3.count_tokens() == 4
assert b2 != b3

print("\nplay_on_copy demo:")
print("b2:")
print(b2)
print("b3 = b2.play_on_copy(4):")
print(b3)

## ♊ Mirroring Boards

`Board.mirror()` reflects the position horizontally:

- Column 0 ↔ 6  
- Column 1 ↔ 5  
- Column 2 ↔ 4  
- Column 3 stays (center)

This is handy for:
- symmetry reduction,
- debugging evaluation consistency,
- comparing "left vs right" variants of the same tactical motif.


In [ ]:
import bitbully as bb

# Asymmetric position: bottom row filled from left to center.
b = bb.Board()
assert b.play("0123")
m = b.mirror()

show("Original:", b)

show("Mirrored:", m)

# Mirror twice -> same position again (involutive).
mm = m.mirror()
assert mm == b
assert mm.uid() == b.uid()
assert hash(mm) == hash(b)

# A symmetric position should equal its mirror.
sym = bb.Board([1, 3, 5])
sym_m = sym.mirror()
assert sym == sym_m
assert sym.uid() == sym_m.uid()

show("Symmetric example (equals its mirror):", sym)

## 🆔 `Board.uid()`: stable position identifier

`Board.uid()` returns a compact, stable identifier for the **current position**.

Useful properties:
- Two boards with the **same position** have the **same** `uid()`.
- After a move, the `uid()` changes.
- You can use `uid()` as a key for caching evaluations (e.g. a mini transposition table).

This section demonstrates:
1) equality ⇒ equal `uid()`
2) mutation ⇒ different `uid()`


In [ ]:
import bitbully as bb

agent = bb.BitBully()

b1 = bb.Board("33333111")
b2 = bb.Board()
assert b2.play("31133331")  # Different move order, same final position.

print("b1 == b2:", b1 == b2)
print("b1.uid():", b1.uid())
print("b2.uid():", b2.uid())
assert b1.uid() == b2.uid()

# After a move: uid changes
b3 = b1.copy()
assert b3.play(0)
print("\nAfter playing column 0 on a copy:")
print("b3 == b1:", b3 == b1)
print("b3.uid():", b3.uid())
print("b1.uid():", b1.uid())
assert b3.uid() != b1.uid()

## ✅ Legal moves vs. non-losing moves (and ordering)

`Board.legal_moves()` can answer three slightly different questions:

- **All legal moves**: any non-full column.
- **Non-losing legal moves**: filters moves that allow the opponent to win immediately next turn.
- **Ordered moves**: moves are returned in a heuristic order (usually center-first).

This is super useful for:
- quickly identifying forced defense,
- building human-readable analysis,
- and improving search performance with better move ordering.

In [ ]:
import bitbully as bb

# Empty board: all columns are legal.
b = bb.Board()
assert b.legal_moves() == list(range(7))

# Ordered should still contain the same set, but in a more promising order.
ordered = b.legal_moves(order_moves=True)
assert set(ordered) == set(range(7))
print("Ordered legal moves on empty board:", ordered)

# Example position where only one move avoids immediate loss.
b = bb.Board()
assert b.play("3322314")
print("\nPosition:")
print(b)

all_legal = b.legal_moves()
non_losing = b.legal_moves(non_losing=True)

print("\nAll legal moves:      ", all_legal)
print("Non-losing legal moves:", non_losing)

# In this position, the engine reports only one non-losing move.
assert non_losing == [5]


## 🧱 Illegal moves & exceptions (Board + BitBully)

In practice you often need to guard against illegal inputs:

- **Board-level checks**
  - `board.is_legal_move(col)` is a fast legality test.
  - `board.play(col)` returns `False` if the move is illegal (invalid column or full column).

- **Agent-level checks**
  - `agent.score_move(board, col)` *raises* a `ValueError` if the move is illegal.

This section demonstrates a recommended pattern:
1) Use `is_legal_move()` (or `legal_moves()`) to pre-filter
2) Handle failures cleanly for robustness (especially in notebooks / scripts).


In [ ]:
agent = bb.BitBully()
board = bb.Board(6 * "3")  # Fill column 3 completely.

show("Board with full center column (3):", board)

# Board-level legality checks
print("\nBoard legality checks:")
print("is_legal_move(3):", board.is_legal_move(3))
print("is_legal_move(2):", board.is_legal_move(2))
print("is_legal_move(99):", board.is_legal_move(99))  # invalid column index -> False

# Board.play: returns False if illegal
print("\nAttempt board.play(3) (full column):", board.play(3))
print("Attempt board.play(99) (invalid column):", board.play(99))

# Agent.score_move: raises ValueError on illegal moves
print("\nAgent scoring with error handling:")
try:
    _ = agent.score_move(board, column=3)
except ValueError as e:
    print("score_move(board, 3) raised:", e)

## 🏆 Detecting wins: `can_win_next()`

`can_win_next()` checks whether the **current player** has an immediate winning move.

- `can_win_next(col)` checks a specific column
- `can_win_next()` checks if *any* winning move exists

In [ ]:
import bitbully as bb

# Create a board from a move string.
# The string "332311" represents a short sequence of alternating moves
# that results in a nearly winning position for Player 1 (yellow, X).
board = bb.Board("332311")

print(board)

# Player 1 (yellow, X) — who is next to move — can win immediately
# by placing a token in either column 0 or column 4.
assert board.can_win_next(0)
assert board.can_win_next(4)

# However, playing in other columns does not result in an instant win.
assert not board.can_win_next(2)
assert not board.can_win_next(3)

# You can also call `can_win_next()` without arguments to perform a general check.
# It returns True if the current player has *any* winning move available.
assert board.can_win_next()

## 🏆 Detecting wins: `has_win()`

`has_win()` indicates whether a winning configuration already exists on the board.
A typical use is to call it right after a move.

In [ ]:
# Unlike `can_win_next()`, which checks whether the current player *could* win
# on their next move, the `has_win()` method determines whether a winning
# condition already exists on the board.
#
# This method is typically used right after a move to verify whether the game
# has been won.
import bitbully as bb

# Initialize a board from a move sequence.
# The string "332311" represents a position where Player 1 (yellow, X)
# is one move away from winning.
board = bb.Board("332311")

# At this stage, Player 1 has not yet won, but can win immediately
# by placing a token in either column 0 or column 4.
assert not board.has_win()
assert board.can_win_next(0)  # Check column 0
assert board.can_win_next(4)  # Check column 4
assert board.can_win_next()  # General check (any winning move)

# Simulate Player 1 playing in column 4 — this completes
# a horizontal line of four tokens and wins the game.
assert board.play(4)

# Display the updated board to visualize the winning position.
print(board)

# The board now contains a winning configuration:
# Player 1 (yellow, X) has achieved a Connect-4.
assert board.has_win()

## 🎲 Game end states: win vs draw

BitBully provides several “end-of-game” helpers:

- `board.has_win()` → a connect-4 is already present
- `board.is_full()` → no more legal moves
- `board.is_game_over()` → win OR draw
- `board.winner()` → winning player id (only meaningful if `has_win()` is true)

This section demonstrates:
1) a quick win position (`has_win()` becomes true after a move)
2) a draw position (full board without a win)


In [ ]:
import bitbully as bb

# 1) Win demonstration (simple horizontal connect-4)
w = bb.Board("332311")  # from your earlier win demo
print("Before winning move:")
print(w)
assert not w.has_win()
assert w.can_win_next()

# Play winning move (column 4)
assert w.play(4)
print("\nAfter winning move:")
print(w)
assert w.has_win()
assert w.is_game_over()

# winner() should now be meaningful (player who just won)
print("Winner:", w.winner())
assert w.winner() in {1, 2}

# 2) Draw demonstration
d, _ = bb.Board.random_board(42, forbid_direct_win=True)
print("\nFull-board draw position:")
print(d)
print("is_full:", d.is_full())
print("has_win:", d.has_win())
print("is_game_over:", d.is_game_over())
assert d.is_full()
assert not d.has_win()
assert d.is_game_over()

# Board Utilities
## ⚖️ Equality / inequality

Board equality is **position-based** (bit patterns), not "history-based".

That means two different move sequences can still be equal if they lead to the same token layout.


In [ ]:
import bitbully as bb

# Create two boards that should represent *identical* game states.
board1 = bb.Board("33333111")  # Built directly from a move string.

board2 = bb.Board()
# Play the same position step by step using a different but equivalent sequence.
# Internally, the final bitboard state will match `board1`.
board2.play("31133331")

# Boards with identical token placements are considered equal.
# Equality (`==`) and inequality (`!=`) operators are overloaded for convenience.
assert board1 == board2
assert not (board1 != board2)  # noqa: SIM202

# ------------------------------------------------------------------------------

# Create two boards that differ by one move.
board1 = bb.Board("33333111")
board2 = bb.Board("33333112")  # One extra move in the last column (index 2)

# Since the token layout differs, equality no longer holds.
assert board1 != board2
assert not (board1 == board2)  # noqa: SIM201

# ------------------------------------------------------------------------------

# Notes:
# - Equality checks in BitBully compare the *exact board state* (bit patterns),
#   not just the move history.
# - Two different move sequences can still yield the same position if they
#   result in identical token configurations.
# - This is useful for comparing solver states, verifying test positions,
#   or detecting transpositions in search algorithms.

## 🔑 Hashing boards

If two boards represent the same position, their hashes match.
This makes boards usable as keys in dictionaries / members in sets.

In [ ]:
import bitbully as bb

# Create two boards that should represent *identical* game states.
board1 = bb.Board("33333111")  # Built directly from a move string.

board2 = bb.Board()
# Play the same position step by step using a different but equivalent sequence.
# Internally, the final bitboard state will match `board1`.
board2.play("31133331")

assert hash(board1) == hash(board2)
hash(board1)

## 🧠 Scoring moves with BitBully

BitBully can score moves in two ways:

- `score_move(board, col)` → score a **single** move
- `score_all_moves(board)` → scores **all legal moves** and returns a dict `{col: score}`

Notes:
- Higher score is better for the **player to move**.
- Full columns are omitted from `score_all_moves`.
- In the starting position, the center column (3) is typically best.


In [ ]:
import bitbully as bb

agent = bb.BitBully()
board = bb.Board()

scores = agent.score_all_moves(board)
print("Scores on empty board:", scores)

# Single-move scoring agrees with the dict entry.
s3 = agent.score_move(board, column=3)
assert s3 == scores[3]

# The canonical starting ordering used in your docs.
assert scores == {3: 1, 2: 0, 4: 0, 1: -1, 5: -1, 0: -2, 6: -2}

# If a column is full, it is omitted.
full_center = bb.Board(6 * "3")
scores2 = agent.score_all_moves(full_center)
print("\nScores with center column full:", scores2)

assert 3 not in scores2

## 🎯 Choosing a best move (tie-break strategies)

`BitBully.best_move()` resolves ties between equally scoring moves using:

- `"center"` (default): closest to column 3
- `"leftmost"`: smallest column index
- `"random"`: random among best moves (optionally deterministic via `rng=Random(seed)`)

This is useful when:
- multiple moves are equally good,
- you want stable behavior in tests,
- or you want variability for self-play.


In [ ]:
import random

import bitbully as bb

agent = bb.BitBully()

board = bb.Board("341")
show('Board("341"):', board)

scores = agent.score_all_moves(board)
print("Scores:", scores)

# Several moves can be tied as "best" depending on the position.
c_center = agent.best_move(board, tie_break="center")
c_left = agent.best_move(board, tie_break="leftmost")
c_rand = agent.best_move(board, tie_break="random")

print("\nBest (center):  ", c_center)
print("Best (leftmost):", c_left)
print("Best (random):  ", c_rand)

# Reproducible randomness:
rng = random.Random(42)
c_rand2 = agent.best_move(board, tie_break="random", rng=rng)
print("Best (random, rng=42):", c_rand2)

# Sanity checks (match your docstring expectations).
assert c_center == 3
assert c_left == 1
assert c_rand in {1, 3, 4}
assert c_rand2 in {1, 3, 4}

## 🔍 Cross-check: negamax vs null-window vs MTD(f)

BitBully exposes multiple search methods.
A practical sanity-check is to compare them on the same position:

- `negamax(board)`
- `null_window(board)`
- `mtdf(board)`

For typical positions (and same engine settings), these should agree on the returned score.

In [ ]:
import bitbully as bb

agent = bb.BitBully()
board, moves = bb.Board.random_board(n_ply=14, forbid_direct_win=True)

print("Moves:", moves)
show("Board:", board)

score_mtdf = agent.mtdf(board)
score_negamax = agent.negamax(board)
score_null = agent.null_window(board)

print("\nScores:")
print("  mtdf:       ", score_mtdf)
print("  negamax:    ", score_negamax)
print("  null_window:", score_null)

assert score_mtdf == score_negamax == score_null

## 📚 Opening books: available books, loading, and resetting

BitBully can use an opening database automatically (early game),
or you can fully disable it and load books manually.

Typical workflow:
1. List available books via `BitBully.available_opening_books()`
2. Create `BitBully(opening_book=None)` to start without a book
3. Load a book by name (e.g. `"12-ply"`) or by path
4. Reset/unload via `reset_book()`

In [ ]:
import bitbully as bb

books = bb.BitBully.available_opening_books()
print("Available opening books:", books)

# Start without any book loaded.
agent = bb.BitBully(opening_book=None)
assert agent.is_book_loaded() is False
print("Initially loaded?", agent.is_book_loaded())

# Load by identifier (e.g. "default", "8-ply", "12-ply", "12-ply-dist")
agent.load_book("default")
assert agent.is_book_loaded() is True
print("After load_book('default'):", agent)

# Unload it again (search-only mode).
agent.reset_book()
assert agent.is_book_loaded() is False
print("After reset_book():", agent)

## 📈 Measuring search work with the node counter

The engine keeps a node counter (visited nodes) which you can query via:

- `get_node_counter()`
- `reset_node_counter()`

This is handy for:
- comparing move ordering,
- verifying that caching/transposition tables help,
- and measuring the cost of a given query in a reproducible way.

In [ ]:
import bitbully as bb

agent = bb.BitBully()
agent.reset_node_counter()

board = bb.Board()
_ = agent.score_all_moves(board)

nodes = agent.get_node_counter()
print("Nodes visited:", nodes)
assert isinstance(nodes, int)
assert nodes >= 0

agent.reset_node_counter()
assert agent.get_node_counter() == 0

## 🤖 Mini-game: BitBully plays against itself (deterministic)

This section runs a short self-play “mini-game” where the BitBully agent chooses moves for **both players**
from the current position.

Key points:

- We use `rng = random.Random(seed)` to make the game **reproducible**.
- Each turn:
  1. Score all legal moves with `agent.score_all_moves(board)`
  2. Pick a move via `agent.best_move(..., tie_break="random", rng=rng)`
  3. Apply it with `board.play(col)`
- The game stops early if a win is detected (`board.has_win()`) or the board is full.

Adjust:
- `n_moves`: how many plies (half-moves) to play
- `seed`: deterministic randomness for tie-breaking
- `opening_book`: keep default book or disable with `BitBully(opening_book=None)`

In [ ]:
import random

import bitbully as bb


def self_play_minigame(
    *,
    n_moves: int = 12,
    seed: int = 42,
    opening_book: str = "default",
    show_every: int = 1,
) -> tuple[bb.Board, list[int]]:
    """Play a short deterministic self-play mini-game (agent vs itself).

    The agent selects moves for both players by repeatedly calling
    `best_move(..., tie_break="random", rng=rng)` so that tie breaks are
    randomized but reproducible.

    Args:
        n_moves: Number of plies (half-moves) to play from the starting position.
            If the game ends (win or draw) earlier, the loop stops.
        seed: Random seed used for deterministic tie-breaking when multiple moves
            share the best score.
        opening_book: Opening book to use. Use `"default"` for the packaged
            default (typically `"12-ply-dist"`), or `None` to disable book usage.
        show_every: Print the board every `show_every` plies. Use 1 to show every
            move, larger values to reduce output.

    Returns:
        A tuple `(final_board, moves)` where:
        - `final_board` is the resulting position after the played moves.
        - `moves` is the list of played column indices (length <= n_moves).

    Raises:
        ValueError: If `n_moves` is outside [0, 42] or `show_every` < 1.
        RuntimeError: If the agent unexpectedly selects a wrong move.

    Example:
        ```python
        final_board, moves = self_play_minigame(n_moves=8, seed=123, opening_book=None)
        print("Moves:", moves)
        print(final_board)
        ```
    """
    if not 0 <= n_moves <= 42:
        raise ValueError(f"n_moves must be in [0, 42], got {n_moves}.")
    if show_every < 1:
        raise ValueError(f"show_every must be >= 1, got {show_every}.")

    rng = random.Random(seed)
    agent = bb.BitBully(opening_book=opening_book)
    board = bb.Board()
    moves: list[int] = []

    print(f"Self-play mini-game: n_moves={n_moves}, seed={seed}, opening_book={opening_book!r}")
    print("Start position:")
    print(board)

    for ply in range(1, n_moves + 1):
        if board.is_game_over():
            break

        # Score all moves, then pick among the best with deterministic RNG tie-break.
        scores = agent.score_all_moves(board)
        if not scores:
            break  # no legal moves (board full)

        col = agent.best_move(board, tie_break="random", rng=rng)

        ok = board.play(col)
        if not ok:
            raise RuntimeError(f"Engine chose an illegal move unexpectedly: col={col}")

        moves.append(col)

        if ply % show_every == 0 or board.is_game_over() or ply == n_moves:
            player_just_moved = 2 if board.current_player() == 1 else 1
            print(f"\nPly {ply:02d} | Player {player_just_moved} played column {col}")
            print("Scores:", scores)
            print(board)

    # Final result summary
    if board.has_win():
        print(f"\nGame over: Player {board.winner()} wins.")
    elif board.is_full():
        print("\nGame over: Draw (board full).")
    else:
        print("\nStopped early (move limit reached or no legal moves).")

    return board, moves

In [ ]:
# --- Run a demo mini-game (deterministic) ---
final_board, moves = self_play_minigame(
    n_moves=12,
    seed=42,
    opening_book="default",  # set to None for search-only mode
    show_every=1,
)

print("\nMoves played:", moves)

## ⚡ Transposition table effect (node counter + transposition table)

BitBully uses a **transposition table (TT)** internally to cache evaluated positions.
This can reduce the amount of search work on repeated or related queries.

We can observe this effect using:
- `get_node_counter()` / `reset_node_counter()`
- `reset_transposition_table()`

This section compares:
1) Cold run (TT cleared) → measure nodes for a search call
2) Warm run (TT kept) → run the same call again (often fewer nodes)
3) Clear TT again → nodes typically go back up

Note: exact node counts can depend on engine settings and position.


In [ ]:
import bitbully as bb


def measure_nodes_for_call(agent: bb.BitBully, fn_name: str, board: bb.Board) -> int:
    """Call a BitBully method on a board and return visited nodes.

    Args:
        agent: BitBully agent instance.
        fn_name: Name of the method to call: "mtdf", "negamax", or "null_window".
        board: Board position.

    Returns:
        The node counter value after the call.

    Raises:
        ValueError: If `fn_name` is unknown.

    Example:
        ```python
        import bitbully as bb

        agent = bb.BitBully()
        b = bb.Board("33333111")
        n = measure_nodes_for_call(agent, "mtdf", b)
        assert isinstance(n, int) and n >= 0
        ```
    """
    agent.reset_node_counter()

    if fn_name == "mtdf":
        _ = agent.mtdf(board)
    elif fn_name == "negamax":
        _ = agent.negamax(board)
    elif fn_name == "null_window":
        _ = agent.null_window(board)
    else:
        raise ValueError(f"Unknown fn_name: {fn_name!r}")

    return agent.get_node_counter()


agent = bb.BitBully(opening_book=None)  # focus on search/TT behavior
board = bb.Board("33333111")  # 8 moves (fast enough), somewhat non-trivial

print("Test position:")
print(board)

# Ensure TT cold start
agent.reset_transposition_table()

cold = measure_nodes_for_call(agent, "mtdf", board)
warm = measure_nodes_for_call(agent, "mtdf", board)

# Clear TT again
agent.reset_transposition_table()
cold2 = measure_nodes_for_call(agent, "mtdf", board)

print("\nNode counts (mtdf):")
print("  cold :", cold)
print("  warm :", warm)
print("  cold2:", cold2)

# Usually warm <= cold, but allow equality (depends on engine internals/position).
assert warm <= cold or warm == cold
assert cold2 >= 0

## 📚 Opening book vs search-only (8–10 tokens)

Opening books speed up early-game evaluation by returning precomputed values.

To compare fairly (and keep runtime reasonable), we use a position with **8–10 tokens**:
- Deep enough that “search-only” is non-trivial,
- Still early enough that the opening book is applicable.

We compare:
1) `BitBully(opening_book="12-ply-dist")` (book enabled)
2) `BitBully(opening_book=None)` (search-only)

We’ll print:
- scores
- chosen best move
- node counter (only meaningful for search; book lookups may use few/zero nodes)

Note: the book and search should still agree on *the returned score* in many cases,
but node counts can differ drastically.


In [ ]:
import bitbully as bb


def compare_book_vs_search(board: bb.Board) -> None:
    """Compare move scoring with opening book enabled vs search-only.

    Args:
        board: Board position with ~8-10 tokens (recommended for this demo).

    Example:
        ```python
        import bitbully as bb

        b = bb.Board("33445511")  # 8 tokens
        compare_book_vs_search(b)
        ```
    """
    agent_book = bb.BitBully(opening_book="default")
    agent_search = bb.BitBully(opening_book=None)

    print("Position:")
    print(board)
    print("Tokens:", board.count_tokens())
    assert 8 <= board.count_tokens() <= 10

    # --- Book agent ---
    agent_book.reset_node_counter()
    scores_book = agent_book.score_all_moves(board)
    nodes_book = agent_book.get_node_counter()
    best_book = agent_book.best_move(board, tie_break="center")

    # --- Search-only agent ---
    agent_search.reset_node_counter()
    scores_search = agent_search.score_all_moves(board)
    nodes_search = agent_search.get_node_counter()
    best_search = agent_search.best_move(board, tie_break="center")

    print("\nBook-enabled:")
    print("  is_book_loaded:", agent_book.is_book_loaded())
    print("  scores:", scores_book)
    print("  best_move(center):", best_book)
    print("  nodes:", nodes_book)

    print("\nSearch-only:")
    print("  is_book_loaded:", agent_search.is_book_loaded())
    print("  scores:", scores_search)
    print("  best_move(center):", best_search)
    print("  nodes:", nodes_search)

    # Sanity: best moves should be legal.
    assert board.is_legal_move(best_book)
    assert board.is_legal_move(best_search)


# Use a compact 8-token position (8 plies).
# Chosen to be early-game-ish and typically covered by opening books.
board_8, _ = bb.Board.random_board(n_ply=8, forbid_direct_win=True)

compare_book_vs_search(board_8)